# Preparar el espacio de trabajo en Colab

##Crear directorios
Para trabajar de forma ordenada, se crearán 3 carpetas en colab:
- *rawData*: (**input**) 
en donde estarán los archivos necesarios para trabajar 
- *results*: en donde se guardarán los archivos generados (**output**)
- *plots*: en donde se guardarán los pdf de las figuras (**output**)

In [ ]:
#@title Desde el navegador de la derecha, revisar los directorios
system("mkdir /content/rawData", TRUE)
system("mkdir /content/plots", TRUE)
system("mkdir /content/results", TRUE)
system("rm -r /content/sample_data", TRUE) # eliminarmos la carpeta que está por default en colab
system("ls", TRUE) # listamos los elementos del directorio

##Cargar archivos
En el directorio *rawData* cargaremos los inputs



In [ ]:
#@title Descargar archivos en el directorio
setwd("/content/rawData") # establecemos el directorio de trabajo
system("gdown --id 1DaP68_pTodGxY0eIT0B3WEWubbeUj9iL") # archivo del genemapper
system("gdown --id 1sqxFlxuOKEkcuQZbGL_URzNLVnMrJzd7") # funciones
system("ls", TRUE) # listamos los elementos del directorio

# Preparar el espacio de trabajo en R




In [ ]:
#@title Instalar librerias
source(file.path("/content/rawData", "commonFunctions.R"))

In [ ]:
#@title Establecer el directorio de trabajo
setwd("/content/results")
getwd()

# Análisis de datos obtenidos por *MSAP*

## Análisis exploratorio de los datos

In [ ]:
#@title Abrir la matriz de datos en R
Data <- read.csv("/content/rawData/MSAP_Genemapper.csv", header = TRUE, row.names = 1, check.names = FALSE)
numHybrids<-nrow(Data)
numLoci<-ncol(Data)
cat("Número de epiLoci=", numLoci,"\n")
cat("Número de muestras=", numHybrids/2 ,"\n")
cat("\n","\t\t\t\t Vista previa de la tabla:", "\n")
Data[1:5,1:10]

Generar la matriz de patrones comparando *_h* (HpaII) y *_m* (MspaI).

<div id="image-table">
    <table>
      <tr>
          <td style="padding:10px">
              <img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_01_A.png" width="400"/>
            <td style="padding:10px">
            </td>
        </tr>
    </table>
</div> 

In [ ]:
#@title
MSAP_patterns<- matrix(nrow = dim(Data)[2], ncol = 0)
out<- c()
for (hyb in 1:numHybrids) if (hyb %% 2 ==1){
  for (loci in 1:numLoci){
    out<- append(out, ifelse(Data[hyb,loci]+Data[hyb+1,loci]==0,'0', # en este caso, suma los valores de las fila j y j+1 en la fila i
                      # si el test es falso, continua con la siguiente línea que es otro test
                      ifelse(Data[hyb,loci]+Data[hyb+1,loci]==2,'1',
                      # si el test es falso, termina con el 3
                      ifelse(Data[hyb,loci]+Data[hyb+1,loci]==1,'2','3'))))
  }
  MSAP_patterns<-cbind(MSAP_patterns,matrix(out,dimnames = list(colnames(Data),substr(row.names(Data)[hyb],1,nchar(as.character(row.names(Data)[hyb]))-2))))
  out<- c()
}
cat("Número de epiLoci=", nrow(MSAP_patterns),"\n")
cat("Número de muestras=", ncol(MSAP_patterns),"\n")
cat("\n","\t\t\t\t Vista previa de la tabla:", "\n")
MSAP_patterns[1:5,]
#guardamos
write.csv(t(MSAP_patterns), file = "/content/results/MSAP_patterns.csv")

## Selección de alelos polimórficos
Solo los alelos polimorficos brindan información.

Descartaremos alelos que sean:
- **Singletons**: aquellos que están presentes en una sola muestra.
- **Monomorficos**: aquellos que están presentes en todas las muestras.

<div id="image-table">
    <table>
      <tr>
          <td style="padding:10px">
              <img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_01_B.png" width="400"/>
            <td style="padding:10px">
              <img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_01_C.png" width="400"/>
            </td>
        </tr>
    </table>
</div> 

In [ ]:
#@title Generamos las variables dummy para cada patrón.
dummies<- matrix(nrow = ncol(MSAP_patterns), ncol = 0)

for (loci in 1:numLoci){
  dummies <- cbind(dummies,
          matrix(as.numeric(MSAP_patterns[loci,] == 1), dimnames = list(colnames(MSAP_patterns), paste(row.names(MSAP_patterns)[loci],"1",sep = "_"))),
          matrix(as.numeric(MSAP_patterns[loci,] == 2), dimnames = list(colnames(MSAP_patterns), paste(row.names(MSAP_patterns)[loci],"2",sep = "_"))),
          matrix(as.numeric(MSAP_patterns[loci,] == 3), dimnames = list(colnames(MSAP_patterns), paste(row.names(MSAP_patterns)[loci],"3",sep = "_"))))
}

In [ ]:
#@title Filtrado y selección de polimórficos
# Por cada alelo, vamos a sumar el número de muestras que presentan el alelo.
temp<-data.frame(colSums(dummies, dims=1))
colnames(temp)<-c("count")
# Definimos las variables
singletons<-1
monomorphic<-nrow(dummies)
toKeep<-subset(temp, c(count>singletons & count<monomorphic))
toKeep<-rownames(toKeep) #alelos polimorficos
#seleccionamos solo polimorficos de la matriz dummies
binary_matrix<-dummies[,toKeep]
cat("Número de alelos monomorficos =", sum(temp$count==nrow(dummies)), "\n")
cat("Número de alelo singletons =", sum(temp$count==1), "\n")
#cat("Número de alelo nulo =", sum(temp$count<1), "\n")
cat("Número de alelo polimorficos =", dim(binary_matrix)[2], "\n")
cat("Número de alelo totales =", sum(temp$count>=1), "\n")
cat("Número de muestras =", nrow(binary_matrix), "\n")
cat("\n","\t\t\t\t Vista previa de la tabla:", "\n")
binary_matrix[1:5,1:5]
# Guardamos la matriz binaria
write.csv(binary_matrix, file = "/content/results/MSAP_binary_matrix.csv")

## Metilación global

In [ ]:
#@title Conteo de alelos metilados y no metilados
conteoPatrones<-f.count.methylation(binary_matrix)
head(conteoPatrones)
write.csv(conteoPatrones, file = "patrones.csv")
# Calculamos valores medios para cada categoría
summaryPatrones<-f.summary.patrones(conteoPatrones)
write.csv(summaryPatrones, file = "summary_Patrones.csv")

In [ ]:
#@title Gráfico
ggplot(summaryPatrones,aes(x=variable,y=mean, fill=samples))+ geom_bar(stat="identity", position=position_dodge()) +
geom_errorbar(aes(ymin=mean-sd, ymax=mean+sd), width=.2, position=position_dodge(.9)) +
scale_fill_manual(values=c('aquamarine3', 'coral1', 'plum4', 'plum2')) +
labs(title="", x="", y = "Proporción de epialelos")+ theme(axis.text=element_text(size=14),
        axis.title=element_text(size=14,face="bold"))

pdf("/content/plots/methylation.pdf")
ggplot(summaryPatrones,aes(x=variable,y=mean, fill=samples))+ geom_bar(stat="identity", position=position_dodge()) +
geom_errorbar(aes(ymin=mean-sd, ymax=mean+sd), width=.2, position=position_dodge(.9)) +
scale_fill_manual(values=c('aquamarine3', 'coral1', 'plum4', 'plum2')) +
labs(title="", x="", y = "Proporción de epialelos")+ theme(axis.text=element_text(size=14),
        axis.title=element_text(size=14,face="bold"))
dev.off()

## Comparación de híbridos
Vamos a comparar los hibridos sintéticos con los parentales (S. kurtzianum y S. microdontum) y los híbridos naturales (S. x rechei).

Queremos saber cuántos alelos comparte con cada uno.

In [ ]:
#@title Alelos comunes y únicos en el híbrido sintético
numAlleles<-ncol(binary_matrix) # Number of alleles
natHyb<-grep("Rechei", rownames(binary_matrix), value = TRUE) # S. x rechei (Natural hybrid)
synHyb<-grep("Hybrid", rownames(binary_matrix), value = TRUE) # Synthetic hybrids

out<- c()
results<- matrix(nrow = 4, ncol = 0)

## This is a double loop for hyb hybrids and allele loci. Rows are as follows: 1-5 Hybrids, 6 S. kurtzianum (Parental1),
## 7 S. microdontum (Parental2), 8-15 .
for (hyb in synHyb){
  for (allele in 1:numAlleles)
  {
    out<- append(out, ifelse(binary_matrix[hyb,allele]>0 & binary_matrix['Kurtzianum',allele]>0 & binary_matrix['Microdontum',allele]==0 & sum(binary_matrix[natHyb,allele])==0,'A',
                      ifelse(binary_matrix[hyb,allele]>0 & binary_matrix['Kurtzianum',allele]==0 & binary_matrix['Microdontum',allele]>0 & sum(binary_matrix[natHyb,allele])==0,'B',
                      ifelse(binary_matrix[hyb,allele]>0 & binary_matrix['Kurtzianum',allele]==0 & binary_matrix['Microdontum',allele]==0 & sum(binary_matrix[natHyb,allele])>0,'C',
                      ifelse(binary_matrix[hyb,allele]>0 & binary_matrix['Kurtzianum',allele]==0 & binary_matrix['Microdontum',allele]==0 & sum(binary_matrix[natHyb,allele])==0,'D','E')))))
   }                         
  
  results<-cbind(results,matrix(table(out, exclude = "E"),dimnames = list(c("Kurtzianum","Microdontum","Rechei","Unique"),hyb)))
  #results<-cbind(results,matrix(table(out, exclude = "E"),dimnames = list(c("Kurtzianum","Microdontum","Rechei","Hybrid"),hyb)))
  out<- c()
}  

## Transform counts by hybrid into percentages 
percentages<-t(results)/colSums(results)*100
head(percentages)
## Save output to working directory
write.csv(percentages, file = "/content/results/MSAP_Hybrids.csv")

In [ ]:
#@title Cálculo de valores medios
percentagesToPlot<-melt(percentages)
colnames(percentagesToPlot)<-c("Hybrids","Especies","value")
summaryComparacion<-ddply(percentagesToPlot, c("Especies"), summarise,
               mean = mean(value),
               sd = sd(value))
write.csv(summaryComparacion, file = "ComparacionHibridos.csv")
summaryComparacion

In [ ]:
#@title Gráfico
ggplot(summaryComparacion,aes(x=Especies,y=mean, fill=Especies))+ geom_bar(stat="identity", position=position_dodge(), show.legend = F) +
geom_errorbar(aes(ymin=mean-sd, ymax=mean+sd), width=.2, position=position_dodge(.9)) +
scale_fill_manual(values=c('aquamarine3', 'coral1', 'plum4', 'plum2')) +
labs(title="", x="", y = "Epi-alelos compartidos \n (%)")+ theme(axis.text=element_text(size=14),
        axis.title=element_text(size=14,face="bold"))
pdf("/content/plots/comparaciónHibridos.pdf")
ggplot(summaryComparacion,aes(x=Especies,y=mean, fill=Especies))+ geom_bar(stat="identity", position=position_dodge(), show.legend = F) +
geom_errorbar(aes(ymin=mean-sd, ymax=mean+sd), width=.2, position=position_dodge(.9)) +
scale_fill_manual(values=c('aquamarine3', 'coral1', 'plum4', 'plum2')) +
labs(title="", x="", y = "Epi-alelos compartidos \n (%)")+ theme(axis.text=element_text(size=14),
        axis.title=element_text(size=14,face="bold"))
dev.off()

# Guardar en Drive







Se necesita establecer conexión con Drive para obtener el material de trabajo.
Para ello, se utilizará el paquete *googledrive*. 

In [ ]:
#@title Instalar, cargar y configurar el paquete
install.packages(c("googledrive","R.utils","httr"))

#@title Carga los paquetes
suppressPackageStartupMessages({
  library("googledrive")
  library("R.utils")
  library("httr")
})

#@title Autentificar usuario
my_check <- function() {return(TRUE)}
reassignInPackage("is_interactive", pkgName = "httr", my_check) 
options(rlang_interactive=TRUE)

En el siguiente paso, se debe verificar el usuario.

El siguiente código, va a generar dos cosas:
- 1º: Un link para darle acceso al paquete a nuestra cuenta de Drive. Una vez completado los permisos, va a generar un link que hay que **copiar**
- 2º: Un cuadro de texto con el código de autorización: **pegar** el link del primer paso.

In [ ]:
#@title Otorgar permisos
drive_auth(use_oob = TRUE, cache = TRUE)

Una vez que estamos autorizados a editar Drive, vamos a guardar todos los arhivos que están en colab.

In [ ]:
#@title Guardar archivos en el Drive personal
folder2save<-dir('/content', full.names = TRUE)
file2save<-c()
for (f in folder2save){
  file2save<-dir(f, full.names=TRUE)
  for(i in file2save){
    data1 <- drive_upload(i,"~/EpiEvo/01_TP/", overwrite = TRUE) #descarga a cada carpeta personal
  }
}
